In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from copy import deepcopy
import time
import pickle

In [2]:

def top_patients(NUM_patients=1,MODEL='feature_matrix_model2_stage1.csv'):
    df = pd.read_csv(MODEL).sort_values(['max_malig'],ascending=[False])
    top_patients_dict = {}
    for i in range(NUM_patients):
        
        patient = df.iloc[i]['Unnamed: 0'][:-4]
        top_patients_dict[patient] = {}
        
        malignancy = df.iloc[i]['max_malig']
        top_patients_dict[patient]['max_malig'] = malignancy
        
        print ('Patient',i+1,':\t',patient,'\nMalignancy',i+1,':\t',malignancy)
        
        with open('./LUNA_model_v2/dict_'+patient+'.pickle', 'wb') as handle:
            pickle.dump(top_patients_dict,handle,protocol=pickle.HIGHEST_PROTOCOL)
        print ('\tDictionary SAVED for..',patient)
        
    return top_patients_dict


In [3]:

top_patients_dict = top_patients()


Patient 1 :	 07bca4290a2530091ce1d5f200d9d526 
Malignancy 1 :	 0.534115552902
	Dictionary SAVED for.. 07bca4290a2530091ce1d5f200d9d526
Patient 2 :	 5a42f0a0d1e060531c20d04ed23efc02 
Malignancy 2 :	 0.53409910202
	Dictionary SAVED for.. 5a42f0a0d1e060531c20d04ed23efc02


In [4]:

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.models import load_model

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.15
set_session(tf.Session(config=config))


Using TensorFlow backend.


In [5]:

def top_patients_predict(top_patients_DICT,MODEL,TOP=1):
    top_patients_dict = deepcopy(top_patients_DICT)
    for patient in top_patients_dict.keys():
        print ('Patient..',patient)
        patient_load = np.load('../data/stage1_voxels_mask/'+patient+'.npz')
        voxels = patient_load['vox']
        print ('\tNumber of voxels to predict..',voxels.shape[0])
        
        preds = np.array(MODEL.predict(x=voxels,batch_size=5))
        top_patients_dict[patient]['preds'] = preds
        np.save('./LUNA_model_v2/preds_'+patient+'.npy',preds)
        print ('\tVoxels predicted..',len(preds))
        
        top_ixs = np.argsort(preds[0],axis=0)[-TOP:]
        top_ixs = [i[0] for i in top_ixs]
        top_patients_dict[patient]['top_ixs'] = top_ixs
        print ('\tNumber of top voxels for visualization..',len(top_ixs))
        
        top_patients_dict[patient]['top_voxels'] = np.vstack([voxels[i] for i in top_ixs])
        with open('./LUNA_model_v2/dict_top_patients_predict.pickle', 'wb') as handle:
            pickle.dump(top_patients_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print ('\tDictionary SAVED for..',patient)
        
    return top_patients_dict


In [6]:

LUNA_model_v2 = load_model('../LungCancer/Models/LUNA_model_v2.h5')
top_patients_dict = top_patients_predict(top_patients_dict,LUNA_model_v2,TOP=1)


/opt/anaconda/lib/python3.5/site-packages/keras/engine/topology.py:1206: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Patient.. 07bca4290a2530091ce1d5f200d9d526
	Number of voxels to predict.. 472
	Voxels predicted.. 4
	Number of top voxels for visualization.. 1
	Dictionary SAVED for.. 07bca4290a2530091ce1d5f200d9d526


In [7]:

def top_voxels_predict(top_patients_DICT,MODEL):
    top_patients_dict = deepcopy(top_patients_DICT)
    for patient in top_patients_dict.keys():
        print ('Patient..',patient)
        
        top_voxels = top_patients_dict[patient]['top_voxels']
        
        for i in range(top_voxels.shape[0]):
            print ('\tPredicting voxel',i+1)
            start = time.time()
            
            voxel = np.squeeze(top_voxels[i])
            count = 0
            preds_top_voxels = []
            
            for e in np.nditer(voxel,op_flags=['readwrite']):
                e_original = e.copy()
                e[...] = 0
                preds = MODEL.predict(x=np.expand_dims(np.expand_dims(voxel,axis=0),axis=0),batch_size=1)
                preds = [p[0][0] for p in preds]
                preds_top_voxels.append(preds)
                e[...] = e_original
                count +=1
                if count%1000==0:
                    print ('\t\tOut of',64*64*64,',',count,'are done in',time.time()-start)
                    start = time.time()
            top_patients_dict[patient][i+1] = preds_top_voxels
        
        with open('./LUNA_model_v2/dict_top_voxels_predict.pickle', 'wb') as handle:
            pickle.dump(top_patients_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print ('\tDictionary SAVED for..',patient)
        
    return top_patients_dict


In [8]:

top_patients_dict = top_voxels_predict(top_patients_dict,LUNA_model_v2)


Patient.. 07bca4290a2530091ce1d5f200d9d526
	Predicting voxel 1
		Out of 262144 , 1000 are done in 13.391921043395996
		Out of 262144 , 2000 are done in 12.25285530090332
		Out of 262144 , 3000 are done in 13.332795143127441
		Out of 262144 , 4000 are done in 13.787899017333984
		Out of 262144 , 5000 are done in 13.63324761390686
		Out of 262144 , 6000 are done in 13.795911312103271
		Out of 262144 , 7000 are done in 13.718749046325684
		Out of 262144 , 8000 are done in 14.14165472984314
		Out of 262144 , 9000 are done in 12.427061557769775
		Out of 262144 , 10000 are done in 13.88233494758606
		Out of 262144 , 11000 are done in 13.47255563735962
		Out of 262144 , 12000 are done in 13.882982015609741
		Out of 262144 , 13000 are done in 13.407175302505493
		Out of 262144 , 14000 are done in 13.370752573013306
		Out of 262144 , 15000 are done in 13.732171058654785
		Out of 262144 , 16000 are done in 11.773198127746582
		Out of 262144 , 17000 are done in 10.873040199279785
		Out of 262144 

KeyError: 'top_voxels'